In [8]:
import pandas as pd
import numpy as np
import joblib

# ai 날씨 데이터 합치기

In [9]:
people = ["남자", "노년", "분노"]
weather = ["흐림", "비옴"]
total_data = people + weather

# 자료 원하는형식으로 만들기

In [10]:
x = ["남자", "노년", "흐림", "비옴", "분노"]

## 데이터 프레임의 값 채워 넣기(custom데이터 만들기)

In [11]:
def fill_value(total_data):
    
    '''
    새로운 고객 데이터를 더미화 및 표준화
    '''
   

    data_d = pd.DataFrame(columns = ['남자', '여자', '노년', '중장년', '청년', '청소년', '분노', '우울', '중립', '행복',
                                     '구름많음','맑음', '흐림', '주말', '평일', '아침', '저녁', '점심', '비안옴', '비옴'],
                          index=['0']).fillna(0)
    
    display(data_d)
    # AI모델 결과값과 날씨 API 데이터 추가
    
    for i in total_data:
        data_d[total_data] = 1
        
        
    # 시간
    from datetime import datetime
    now = datetime.now()
    now.hour
    
    if now.hour>=16 and now.hour<=23:
        data_d["저녁"] = 1
    elif now.hour>=11 and now.hour<16:
        data_d["점심"] = 1
    else:
        data_d["아침"] = 1
    
    # 요일 
    now.weekday()
    weekday={0:"월",1:"화",2:"수",3:"목",4:"금",5:"토",6:"일"}
    week=weekday[now.weekday()]
    
    if week=="일" or  week=="토":
        data_d["주말"] = 1
    else:
        data_d["평일"] = 1   
#     return data_d
    # 표준화 모델 불러오기
    sd=joblib.load('data/scalemodel.pkl')
    data_sd=pd.DataFrame(sd.transform(data_d))
    
    # 표준화 된 데이터 프레임 컬럼 rename
    data_sd.rename(columns={0:'남자',1:'여자',2:'노년',3:'중장년',4:'청년',5:'청소년',6:'분노',7:'우울',8:'중립',
               9:'행복',10:'구름많음',11:'맑음',12:'흐림',13:'주말',14:'평일',15:'아침',16:'저녁',17:'점심',18:"비안옴",19:"비옴"},inplace=True)
  
    return data_sd.iloc[0]
    
        

In [12]:
customer_data=fill_value(x)
customer_data

,남자,여자,노년,중장년,청년,청소년,분노,우울,중립,행복,구름많음,맑음,흐림,주말,평일,아침,저녁,점심,비안옴,비옴
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


남자      0.839317
여자     -0.839317
노년      2.723968
중장년    -0.521443
청년     -0.806834
청소년    -0.613039
분노      4.599771
우울     -0.480619
중립     -0.638770
행복     -0.955843
구름많음   -0.332893
맑음     -0.889657
흐림      1.086898
주말     -0.709628
평일      0.709628
아침     -0.375395
저녁      0.915657
점심     -0.705847
비안옴    -1.162736
비옴      1.162736
Name: 0, dtype: float64

# 추천 모델

In [13]:
def cluster_result(customer_data):
    
    '''
    우리가 만든 kmean 모델 불러와서 가장 가까운 군집 판별
    '''
    # 군집모델 불러오기
    import joblib
    model=joblib.load('data/model.pkl')
     # 군집모델로 가까운 군집 색출
    cluster_value=model.predict(customer_data.values.reshape(1, -1))[0]
    
    return cluster_value

In [14]:
cluster_value=cluster_result(customer_data)
cluster_value

0

In [47]:
def recomend_Top3(total_data):
    '''
    판별된 군집의 음식 top3 추천
    '''
    #정의된 함수 불러오기
    customer_data=fill_value(total_data)
    cluster_value=cluster_result(customer_data)
    
    # 군집 label과 음식데이터 확률이 들어있는 dict 불러오기
    import json
    cluster_value=str(cluster_value)
    with open("data/recommend_group.json", "r", encoding='utf-8') as json_file:
        json_data = json.load(json_file)
    # top3 음식 반환
    tmp = sorted(json_data['word'][cluster_value], key= lambda x : json_data['word'][cluster_value][x], reverse=True)[0:3]
    
    
    return sorted(tmp, reverse=True) 

In [48]:
recomend_Top3(total_data)

,남자,여자,노년,중장년,청년,청소년,분노,우울,중립,행복,구름많음,맑음,흐림,주말,평일,아침,저녁,점심,비안옴,비옴
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


['초밥', '불고기', '돈가스']